In [43]:
from selenium import webdriver    

## Render page and retrieve window size
Insantiate a Chrome webdriver and load the page. Adjust the window size of the webdriver to be the size of the rendered page to avoid clipping content from view.

In [44]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
driver = webdriver.Chrome(
    executable_path="../bin/chromedriver",
    options=chrome_options)
url = 'http://localhost:3000/margins.html'
driver.get(url)  # must be running a localserver in the viz/ dir
rendered_width = driver.execute_script("return document.documentElement.scrollWidth")
rendered_height = driver.execute_script("return document.documentElement.scrollHeight")
driver.set_window_size(rendered_width,rendered_height)
print(f"window dims H:{rendered_height} X W:{rendered_width}")   

window dims H:600 X W:810


## Plot size and margins
Using the ['margin convention'](https://observablehq.com/@d3/margin-convention), the plot margins are stored in an object.  The width and height are declared in a way that we can conceptually think of them as the entire drawing area of the plot _minus_ the margins. 
```
// .js code
var margin = { top: 20, right: 20, bottom: 100, left: 80 },
width = 800 - margin.left - margin.right,
height = 400 - margin.top - margin.bottom;
```

## svg element
The entire plot will be drawn within an `svg` elmement. This element is scaled to the `width=800` and `height=400`.  Notice again that we use the relationship of the width and height to the margins by adding them when we set the respective attributes.  

This practice enables us to set the height & width margins according to our original size and margin specification in one location and then allows us to refer to it throughout the visualization without explicitly setting the values `800` and `400` _i.e.,_ avoid using "magic numbers".
```
// .js code
var svg = d3.select("body")
    .append("svg")
    .attr("id", "container")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.bottom + margin.top);
```
(The retrieved dimensions of the `svg` account for a 1 px border)

In [45]:
svg = driver.find_element_by_id("container")
svg_src = svg.get_attribute("outerHTML")
svg_transform = svg.value_of_css_property("transform")
print(f"source: {svg_src}")
print(f"svg dims H:{svg.size['height']} X W:{svg.size['width']}")
print(f"svg transform: {svg_transform}")

source: <svg id="container" width="800" height="400"><circle id="origin" cx="0" cy="0" r="5" fill="black"></circle><g id="plot" transform="translate(20,80)"></g></svg>
svg dims H:402 X W:802
svg transform: none


## Group element layout and sizing
A grouping `g` element is used to hold the plot elements.  Since the origin is oriented at the top-left, this element is translated using the top and left margin values.
```
// .js code
var plot = svg.append("g")
    .attr("id","plot")
    .attr("transform", "translate("+margin.left+","+margin.top+")");
```
The translation of the `g` element can be retrieved using a CSS `transform` lookup.  The last two values in the matrix represent the `xy` transform.

In [46]:
g = driver.find_element_by_id("plot")
g_src = g.get_attribute("outerHTML")
g_transform = g.value_of_css_property("transform")
print(f"source: {g_src}")
print(f"group dims H:{g.size['height']} X W:{g.size['width']}")
print(f"group transform: {g_transform}")

source: <g id="plot" transform="translate(20,80)"></g>
group dims H:0 X W:0
group transform: matrix(1, 0, 0, 1, 20, 80)
